# Work on dataset CF_pairs_ahead_behind_file'+str(index)+'.csv'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import sys

from statistics import mean, stdev
from matplotlib.backends.backend_pdf import PdfPages

import time

In [2]:
def LoadData(index):
    
    path = '../../data_set/CF_pairs_ahead_behind/CF_pairs_ahead_behind_file'+str(index)+'.csv'
    
    data = pd.read_csv(path)
    
    return data

In [3]:
def LoadGSRecord(index, position_num):
    
    path = './Position'+str(position_num)+'/Position_'+str(position_num)+'_'+str(index)+'.csv'
    
    record = pd.read_csv(path)
    
    return record

In [53]:
def main(file_index, position_num):
    
    dataframe = LoadData(file_index)
    GS_record = LoadGSRecord(file_index, position_num)
    
    speed_std_array = []
    
    for row in GS_record.itertuples():
        
        start = int(row.stable_start_index)
        end = int(row.stable_end_index)
        data_segment = dataframe.iloc[start : end+1]
        
        speed_std = stdev(dataframe.iloc[start: end+1].subject_speed)
        speed_std_array.append(speed_std)
    
    new_GS_record = GS_record.copy()
    new_GS_record['speed_std'] = speed_std_array
    
    #title_string = str(file_index) + ' ' + str(position_num)
    #plt.title(title_string)
    #plt.hist(speed_std_array)
    #plt.grid(linestyle="--", alpha=0.5)
    #plt.xlabel('Speed STD')
    #plt.ylabel('count')
    #plt.show()
    
    return new_GS_record

In [71]:
start_time = time.time()

for i in [100, -100, 0]:
    for j in range(10):
        if j == 0:
            new_GS_record = main(j, i)
        else:
            new_GS_record = new_GS_record.append(main(j, i))

    new_GS_record.to_csv('./new_GS_record_Position_' +str(i)+'.csv', index=False)
        
print('running time:', time.time() - start_time)

running time: 16.225892543792725


In [81]:
one = pd.read_csv('./new_GS_record_Position_100.csv')
two = pd.read_csv('./new_GS_record_Position_-100.csv')
three = pd.read_csv('./new_GS_record_Position_0.csv')

result = one.append(two).append(three)

result.to_csv('./new_GS_record_aggregate.csv', index=False)

In [85]:
def Speed_STD_plot(data, position_num):
    
    if position_num == 'total':
        path =  './GS_Speed_STD_'+position_num+'.jpg'
        
        title_string = position_num
        plt.title(title_string)
        plt.hist(data.speed_std)
        plt.grid(linestyle="--", alpha=0.5)
        plt.xlabel('Speed STD')
        plt.ylabel('count')

        plt.savefig(path, dpi=600)
        plt.close('all')
        
    else:
        path =  './GS_Speed_STD_Position_'+str(position_num)+'.jpg'

        title_string = 'Position Num: ' + str(position_num)
        plt.title(title_string)
        plt.hist(data.speed_std)
        plt.grid(linestyle="--", alpha=0.5)
        plt.xlabel('Speed STD')
        plt.ylabel('count')

        plt.savefig(path, dpi=600)
        plt.close('all')

In [88]:
Speed_STD_plot(one, 100)
Speed_STD_plot(two, -100)
Speed_STD_plot(three, 0)
Speed_STD_plot(result, 'total')

In [95]:
detail = one.speed_std.describe()
detail.to_csv('./Position100_SpeedSTD_Distribution.csv')

detail = two.speed_std.describe()
detail.to_csv('./Position-100_SpeedSTD_Distribution.csv')

detail = three.speed_std.describe()
detail.to_csv('./Position0_SpeedSTD_Distribution.csv')

detail = result.speed_std.describe()
detail.to_csv('./PositionAll_SpeedSTD_Distribution.csv')